# add addl USCDI element extensions and tags to SD

1. get list of data elements that are add'l USCDI
2. read SD from resources-yaml folder
3. create a copy
4. find elements based on the addl USCDI elements list
   1. add USCDI extension
   2. add add'l USCDI tag ( in unicode ) to short
      1. if short does not exist, add short from base resource - todo - not needed for this set
   3. sort keys in element
   4. save copy 

In [5]:
from yaml import load as yload, dump as ydump, Loader
from json import loads, dumps
from pathlib import Path
from copy import deepcopy
from datetime import datetime
from fhir.resources.structuredefinition import StructureDefinition

my_path = r'/Users/ehaas/Documents/FHIR/USCDI4-Sandbox/input'
# my_path = r'/Users/ehaas/Documents/FHIR/US-Core/input'
in_path = Path(my_path) / 'resources-yaml'
out_path = Path(my_path) / 'resources-yaml-copy'
out_path.mkdir(parents=True, exist_ok=True)
#create out_path using pathlib Path.mkdir(parents=True, exist_ok=True)

R4_path = Path(r'/Users/ehaas/.fhir/packages/hl7.fhir.r4.core#4.0.1/package')

today = datetime.today().strftime('%Y-%m-%d')
today, in_path, out_path

('2023-10-17',
 PosixPath('/Users/ehaas/Documents/FHIR/US-Core/input/resources-yaml'),
 PosixPath('/Users/ehaas/Documents/FHIR/US-Core/input/resources-yaml-copy'))

In [6]:
# get csv file of add'l uscdi elements using pandas
import pandas as pd
additional_uscdi = Path(my_path) / 'data/additional-uscdi-requirements.csv'
df = pd.read_csv(additional_uscdi)

#list the FHIR Element column
addl_uscdi_elements = df['FHIR Element'].tolist()
addl_uscdi_elements = [x.split(' or ') for x in addl_uscdi_elements[1:]]
addl_uscdi_elements = [item for sublist in addl_uscdi_elements for item in sublist]
# addl_uscdi_elements = list(set(addl_uscdi_elements))
addl_uscdi_elements

['Patient.telecom',
 'Patient.communication',
 'Patient.extension:race',
 'Patient.extension:ethnicity',
 'Patient.extension:tribalAffiliation',
 'Patient.extension:sex',
 'Patient.extension:genderIdentity',
 'Patient.deceased[x]',
 'Patient.address.use',
 'Patient.address.period',
 'Patient.name.use',
 'Patient.name.period',
 'Patient.name.suffix',
 'ServiceRequest.reasonCode',
 'ServiceRequest.reasonReference',
 'Procedure.reasonCode',
 'Procedure.reasonReference',
 'MedicationRequest.reasonCode',
 'MedicationRequest.reasonReference',
 'MedicationRequest.extension:medicationAdherence',
 'Procedure.basedOn',
 'DocumentReference.category:uscore',
 'Observation.derivedFrom',
 'Specimen.collection.bodySite',
 'Specimen.condition']

In [7]:
def get_shorty(r_type, element_path):
    for sd_base in R4_path.glob(f'StructureDefinition*-{r_type}.json'):
        try:
            r_base = loads(sd_base.read_text())
        except Exception as e:
            print("Exception: {}".format(type(e).__name__))
            print("Exception message: {}".format(e))
        # print(f"======{sd_base}, {r_base['type']}======")
        for element in r_base['differential']['element']:
            if element['id'] == element_path:
                return element['short']
                              
        print(f'❗❗❗element not found in base SD - look at the datatype in the path {element_path}')
        #use the penultimate element in element_path to find the datatype in the base SD
        parent_element_path = element_path.rsplit('.', 1)[0]
        #use the last two element in element_path to find the datatype element in the base SD
        datatype_element_path = element_path.split('.', )[-1]

        print(f"parent_element_path = {parent_element_path} datatype_element_path = {datatype_element_path}")

        #get the datatype
        for element in r_base['differential']['element']:
            if element['id'] == parent_element_path:
                for type in element['type']:
                    print(f"datatype = {type['code']}")
                    #get the datatype SD short and compare to the shorty
                    path = f"{type['code']}.{datatype_element_path}"
                    print(f"type = {type['code']}, path = {path}")
                    return get_shorty(type['code'], (f"{type['code']}.{datatype_element_path}"),)



# read StructureDefinitions from resources-yaml folder using pathlib
for sd in in_path.glob('Struct*.yml'):
    print(f"***********{sd}***********")
    r = yload(sd.read_text(), Loader=Loader)
    r_copy = deepcopy(r)
    print(f"☞ ☞ ☞ Type = {r['type']}")
    for i, element in enumerate(r['differential']['element']):
        try:
            print(f"☞ ☞ ☞ element {i}: {element['id']}")
            if element['id'] in addl_uscdi_elements:
                print('❗❗❗add the USCDI extension')
                r_copy['differential']['element'][i]["extension"] = [{
                    "url": "http://hl7.org/fhir/us/core/StructureDefinition/uscdi-requirement",
                    "valueBoolean": True
                }]
                if 'short' in element.keys():
                    shorty = element['short']
                else:
                    print("❗❗❗add the short element from the base")
                    shorty = get_shorty(r['type'], element['path'])

                print("❗❗❗add the '𝗔𝗗𝗗𝗜𝗧𝗜𝗢𝗡𝗔𝗟 𝗨𝗦𝗖𝗗𝗜:' tag to short")
                r_copy['differential']['element'][i]["short"] = f"𝗔𝗗𝗗𝗜𝗧𝗜𝗢𝗡𝗔𝗟 𝗨𝗦𝗖𝗗𝗜: {shorty}"
                
            print('❗❗❗ done with element')
            print('-'*10)

        except Exception as e:
            print("Exception: {}".format(type(e).__name__))
            print("Exception message: {}".format(e))
            print('-'*10)

    print('❗❗❗ done with SD')
    print('-'*40)
    print('\n')

    # custom sort the keys in the elements using fhir.resources
    r_copy = StructureDefinition.parse_obj(r_copy)


    # save the copy to out_path using Pathlib
    copyfile = out_path / sd.name
    copyfile.write_text(r_copy.yaml(indent=2))

***********/Users/ehaas/Documents/FHIR/US-Core/input/resources-yaml/StructureDefinition-us-core-sex.yml***********
☞ ☞ ☞ Type = Extension
☞ ☞ ☞ element 0: Extension
❗❗❗ done with element
----------
☞ ☞ ☞ element 1: Extension.url
❗❗❗ done with element
----------
☞ ☞ ☞ element 2: Extension.value[x]
❗❗❗ done with element
----------
❗❗❗ done with SD
----------------------------------------


***********/Users/ehaas/Documents/FHIR/US-Core/input/resources-yaml/StructureDefinition-us-core-specimen.yml***********
☞ ☞ ☞ Type = Specimen
☞ ☞ ☞ element 0: Specimen
❗❗❗ done with element
----------
☞ ☞ ☞ element 1: Specimen.identifier
❗❗❗ done with element
----------
☞ ☞ ☞ element 2: Specimen.type
❗❗❗ done with element
----------
☞ ☞ ☞ element 3: Specimen.subject
❗❗❗ done with element
----------
☞ ☞ ☞ element 4: Specimen.collection
❗❗❗ done with element
----------
☞ ☞ ☞ element 5: Specimen.collection.bodySite
❗❗❗add the USCDI extension
❗❗❗add the '𝗔𝗗𝗗𝗜𝗧𝗜𝗢𝗡𝗔𝗟 𝗨𝗦𝗖𝗗𝗜:' tag to short
❗❗❗ done with elemen

In [8]:
%%bash -s "$in_path" "$out_path"
##### DO THIS ONLY when you want to overwrite resources-yaml folder with resources-yaml-copy ####

cp $2/Struct*.yml $1